Done so far :


*   Lemmatization
*   Stop Words Removal



In [10]:
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

####### After importing nltk, run the following only once ######
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('stopwords')

In [11]:
def get_wordnet_pos(word):
    tag=nltk.pos_tag([word])[0][1][0].upper()
    tag_dict={"J": wordnet.ADJ, 
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
    return tag_dict.get(tag,wordnet.NOUN)

def lemma_stop(str):
    lemmatizer = WordNetLemmatizer()
    # sentence=arr[0][4]
    tokenizer=RegexpTokenizer('\w+|\$]\d\[+|\S+,-')
    tokenized = tokenizer.tokenize(str)
    lemmatized=[lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in tokenized]
    stop_words=set(stopwords.words('english'))
    filtered_sentence=[w for w in lemmatized if w.lower() not in stop_words]
    after_lemma_stop=' '.join(w for w in filtered_sentence)
    return filtered_sentence
#     return after_lemma_stop

In [12]:
# loading data.npy
# data.npy is a 2D array containing the dataset information as
# data[i][0] : docID of ith document
# data[i][1] : title of ith document
# data[i][4] : content of ith document

data = np.load('data.npy',allow_pickle=True)
# sentence = data[0][4]
# print(sentence)

In [13]:
# creating a map {docID, index_in_data_npy}

# ex. if ith element in data has docID j,
# get_index[i] will return j

get_index = {}

for i in range(0, len(data)) :
    get_index[i] = int(data[i][0])

In [14]:
# Tester code
import time
from tqdm import tqdm

s = time.time()

for i in tqdm(range(0, len(data))) :
    f_content = data[get_index[i]][4]
    contents = f_content
    # print(contents)
    final = lemma_stop (contents)
#     print(type(final))
    # print (final)
    
print(time.time()-s)

  3%|▎         | 6430/204135 [08:21<5:10:31, 10.61it/s] 

KeyboardInterrupt: 

In [ ]:
import trie

In [ ]:
collection = trie.CollectionNode()
doc1 = trie.Node()
for w in final:
    print (w)
    collection.add_(w, 0, 1)
    doc1.add(w, 0)
#     break

In [ ]:
doc2 = trie.Node()
for w in final:
    collection.add_(w, 0, 2)
    doc2.add(w, 0)

In [ ]:
print(collection.get_doc_list("never",0))

In [ ]:
query = "never cooper"
final_query = lemma_stop(query)
print(final_query)

In [ ]:
tf_query = {}
for w in final_query:
    if w not in tf_query:
        tf_query[w] = 1
    else:
        tf_query[w] += 1
    print(w)
    df = len(collection.get_doc_list(w,0))
    print(collection.get_doc_list(w,0))

In [8]:
from tqdm import tqdm

for i in tqdm(range(100000000)):
    i = i

100%|██████████| 100000000/100000000 [00:18<00:00, 5545911.59it/s]


***Original Text :***

      We’re back in Dale Cooper’s position, wandering through a freshly revived world, and trying to catch up with the ways it’s moved on in his absence.

***Processed Text :***

      We back Dale Cooper position wander freshly revive world try catch way move absence